In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
# Load S&P 500 df
df_quake_gold = pd.read_csv("https://raw.githubusercontent.com/labs13-quake-viewer/ds-data/master/" +
                             "Gold%20Price%20Change%20by%20Earthquake.csv", index_col=0)
df_quake_gold.shape

(1445, 9)

In [3]:
df_quake_gold.head()

,Date,Mag,Price_Day_0,Price_Day_7,Price_Day_14,Price_Day_30,Appr_Day_7,Appr_Day_14,Appr_Day_30
0,1968-04-01,7.5,37.70,37.05,38.00,39.2,-1.724138,0.795756,3.978780
1,1968-04-01,6.8,37.70,37.05,38.00,39.2,-1.724138,0.795756,3.978780
2,1968-05-14,6.7,39.80,42.40,41.50,41.3,6.532663,4.271357,3.768844
3,1968-05-16,8.2,42.25,41.40,41.95,40.7,-2.011834,-0.710059,-3.668639
4,1968-05-16,7.9,42.25,41.40,41.95,40.7,-2.011834,-0.710059,-3.668639


In [0]:
dates = []
for i in df_quake_gold.Date:
  dates.append(int(''.join(c for c in i if c.isdigit())))

In [0]:
df_quake_gold["magg"] = (df_quake_gold["Mag"] * 10).astype(int)

In [0]:
df_quake_gold["dates"] = dates

In [7]:
df_quake_gold.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1445 entries, 0 to 1444
Data columns (total 11 columns):
Date            1445 non-null object
Mag             1445 non-null float64
Price_Day_0     1445 non-null float64
Price_Day_7     1445 non-null float64
Price_Day_14    1445 non-null float64
Price_Day_30    1445 non-null float64
Appr_Day_7      1445 non-null float64
Appr_Day_14     1445 non-null float64
Appr_Day_30     1445 non-null float64
magg            1445 non-null int64
dates           1445 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 135.5+ KB


In [8]:
y = df_quake_gold['Appr_Day_30'].astype(str)
X = df_quake_gold[['dates', 'Mag']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
print("Original shape:", X.shape, "\n")

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Original shape: (1445, 2) 

X_train shape: (1083, 2)
X_test shape: (362, 2)
y_train shape: (1083,)
y_test shape: (362,)


In [9]:
X_train.sample()

,dates,Mag
1330,20150530,7.8


In [0]:
# Instantiate model with various parms
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree',
                    nrounds = 'min.error.idx', num_class = 4,
                    maximize = False, eval_metric = 'merror', eta = .2,
                    max_depth = 16, colsample_bytree = .4, nthread = -1)

In [11]:
%%time
# Train model on training data
xgb.fit(X_train, y_train)

CPU times: user 2min 24s, sys: 539 ms, total: 2min 25s
Wall time: 1min 13s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eta=0.2,
              eval_metric='merror', gamma=0, learning_rate=0.1,
              max_delta_step=0, max_depth=16, maximize=False,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nrounds='min.error.idx', nthread=-1, num_class=4,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [0]:
%%time
# Use xgb's predict method on test data
predictions = xgb.predict(X_test)

In [0]:
# Calculate absolute errors
errors = abs(predictions.astype('float64') - y_test.astype('float64'))

In [14]:
# Print out mean absolute error
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 4.67 degrees.


In [15]:
%%time
# Calculate and display accuracy
accuracy = errors.sum() / y_test.astype('float64').sum()
print("For Gold, Incident Mag >= 6.7 ({} incidents)".format(df_quake_gold.shape[0]))
print("XGB Model Score:", xgb.score(X_train, y_train))
print('XGB Model Predictive Accuracy:', round(accuracy, 2), '%.')

For Gold, Incident Mag >= 6.7 (1445 incidents)
XGB Model Score: 0.1440443213296399
XGB Model Predictive Accuracy: 9.21 %.
CPU times: user 2min 12s, sys: 59.2 ms, total: 2min 12s
Wall time: 1min 7s
